In [43]:
import sys
!{sys.executable} -m pip install hdfs


from os import PathLike
from hdfs import InsecureClient
client = InsecureClient("http://hdfs-nn:9870/", user="anonymous")
from_path ="./DrinkingFountains.csv"
to_path ="/Projeto/Bronze/DrinkingFountains.csv"
client.delete(to_path)
client.upload(to_path, from_path)

'/Projeto/Bronze/DrinkingFountains.csv'

In [44]:
 pip install delta-spark

Note: you may need to restart the kernel to use updated packages.


In [45]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/Projeto/Silver'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()


In [46]:
hdfs_path = "hdfs://hdfs-nn:9000/Projeto/Bronze/DrinkingFountains.csv"

customSchema = StructType([
    StructField("FountainTy", StringType(), True),
    StructField("the_geom", StringType(), True),
    StructField("OBJECTID", StringType(), True),
    StructField("Position", StringType(), True),
    StructField("Collection", StringType(), True),
    StructField("Painted", StringType(), True),
    StructField("GISPROPNUM", StringType(), True),
    StructField("SIGNNAME", StringType(), True),
    StructField("BOROUGH", StringType(), True), 
    StructField("FountainCo", IntegerType(), True),
    StructField("GISOBJID", IntegerType(), True),
    StructField("SYSTEM", StringType(), True),
    StructField("DEPARTMENT", StringType(), True),
    StructField("PARENTID", StringType(), True),
    StructField("DESCRIPTIO", StringType(), True),
    StructField("FEATURESTA", StringType(), True),

])
Fontes = spark \
            .read\
            .option("delimiter",",")\
            .option("header","true")\
            .schema(customSchema) \
            .csv(hdfs_path)
Fontes.show()
Fontes.printSchema()

+------------+--------------------+--------+--------------------+--------------------+-------+----------+--------------------+-------+----------+---------+------------+----------+--------+--------------------+----------+
|  FountainTy|            the_geom|OBJECTID|            Position|          Collection|Painted|GISPROPNUM|            SIGNNAME|BOROUGH|FountainCo| GISOBJID|      SYSTEM|DEPARTMENT|PARENTID|          DESCRIPTIO|FEATURESTA|
+------------+--------------------+--------+--------------------+--------------------+-------+----------+--------------------+-------+----------+---------+------------+----------+--------+--------------------+----------+
|  F High Low|POINT (-73.986591...|    1589|         Out in Open|02/10/2018 12:00:...|     no|      B100|Seth Low Playgrou...|      B|         2|100038957| B100-DF0647|      B-11|    B100|F High Low, Out i...|    Active|
|           C|POINT (-73.968627...|    1921|       In Playground|02/22/2018 12:00:...|     no|      M158|Robert Mose

In [47]:
replaced_Fontes = Fontes.drop("PARENTID")
replaced_Fontes2 = replaced_Fontes.drop("FEATURESTA")
replaced_Fontes2.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,BOROUGH,FountainCo,GISOBJID,SYSTEM,DEPARTMENT,DESCRIPTIO
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,B,2,100038957.0,B100-DF0647,B-11,"F High Low, Out in Open"
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,M,1,100039303.0,M158-DF0068,M-06,"C, In Playground"
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,M,1,100039632.0,M011-DF0209,M-04,"D, Under Tree, Near Ballfield, Just Outside Pl..."
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,M,1,100039936.0,M144-DF0392,M-03,"D, Just Outside Playground, Near Ballfield"
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,None,B302,Charlie's Place,B,1,100040304.0,B302-DF0897,B-03,"C, In Shade, In Playground"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,B,1,100039874.0,B018-DF0817,B-18,"D, Out in Open, Near Ballfield"
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Q,1,100040158.0,Q012-DF0750,Q-11,"C, Just Outside Playground"
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,None,11/30/1899 12:00:00 AM +0000,None,X148L1,Virginia Park,X,1,100040464.0,X148L1-DF0470,X-09,A
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,M,1,100039941.0,M144-DF0371,M-03,"E Wheelchair, Near Ballfield"


In [48]:
replaced_Fontes3 = replaced_Fontes2.withColumn('Data', split(replaced_Fontes2['Collection'],' ').getItem(0))
replaced_Fontes3.toPandas()

replaced_Fontes4 = replaced_Fontes3.withColumn('Year', split(replaced_Fontes3['Data'],'/').getItem(2))
replaced_Fontes4.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,BOROUGH,FountainCo,GISOBJID,SYSTEM,DEPARTMENT,DESCRIPTIO,Data,Year
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,B,2,100038957.0,B100-DF0647,B-11,"F High Low, Out in Open",02/10/2018,2018
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,M,1,100039303.0,M158-DF0068,M-06,"C, In Playground",02/22/2018,2018
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,M,1,100039632.0,M011-DF0209,M-04,"D, Under Tree, Near Ballfield, Just Outside Pl...",02/27/2018,2018
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,M,1,100039936.0,M144-DF0392,M-03,"D, Just Outside Playground, Near Ballfield",04/04/2018,2018
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,None,B302,Charlie's Place,B,1,100040304.0,B302-DF0897,B-03,"C, In Shade, In Playground",05/29/2018,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,B,1,100039874.0,B018-DF0817,B-18,"D, Out in Open, Near Ballfield",03/05/2018,2018
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Q,1,100040158.0,Q012-DF0750,Q-11,"C, Just Outside Playground",05/01/2018,2018
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,None,11/30/1899 12:00:00 AM +0000,None,X148L1,Virginia Park,X,1,100040464.0,X148L1-DF0470,X-09,A,11/30/1899,1899
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,M,1,100039941.0,M144-DF0371,M-03,"E Wheelchair, Near Ballfield",04/04/2018,2018


In [49]:
replaced_Fontes5 = replaced_Fontes4.withColumn(
    "Position",
    when(
        (col("Position").isNull()), 
        "Desconhecido"
    ).otherwise(col("Position")))


replaced_Fontes6 = replaced_Fontes5.withColumn(
    "Painted",
    when(
        (col("Painted").isNull()), 
        "Desconhecido"
    ).otherwise(col("Painted")))


replaced_Fontes7 = replaced_Fontes6.withColumn(
    "GISPROPNUM",
    when(
        (col("GISPROPNUM").isNull()), 
        "Indefinido"
    ).otherwise(col("GISPROPNUM")))

In [50]:

replaced_Fontes8 = replaced_Fontes7.withColumn(
    "BOROUGH",
    when(replaced_Fontes7.BOROUGH.endswith('B'),regexp_replace(replaced_Fontes7.BOROUGH,'B','Brooklyn')) \
   .when(replaced_Fontes7.BOROUGH.endswith('M'),regexp_replace(replaced_Fontes7.BOROUGH,'M','Manhattan')) \
   .when(replaced_Fontes7.BOROUGH.endswith('Q'),regexp_replace(replaced_Fontes7.BOROUGH,'Q','Queens')) \
   .when(replaced_Fontes7.BOROUGH.endswith('R'),regexp_replace(replaced_Fontes7.BOROUGH,'R','Staten Island')) \
   .when(replaced_Fontes7.BOROUGH.endswith('X'),regexp_replace(replaced_Fontes7.BOROUGH,'X','Bronx')) \
)



In [51]:
replaced_Fontes8.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,BOROUGH,FountainCo,GISOBJID,SYSTEM,DEPARTMENT,DESCRIPTIO,Data,Year
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,Brooklyn,2,100038957.0,B100-DF0647,B-11,"F High Low, Out in Open",02/10/2018,2018
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,Manhattan,1,100039303.0,M158-DF0068,M-06,"C, In Playground",02/22/2018,2018
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,Manhattan,1,100039632.0,M011-DF0209,M-04,"D, Under Tree, Near Ballfield, Just Outside Pl...",02/27/2018,2018
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,Manhattan,1,100039936.0,M144-DF0392,M-03,"D, Just Outside Playground, Near Ballfield",04/04/2018,2018
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,Desconhecido,B302,Charlie's Place,Brooklyn,1,100040304.0,B302-DF0897,B-03,"C, In Shade, In Playground",05/29/2018,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,Brooklyn,1,100039874.0,B018-DF0817,B-18,"D, Out in Open, Near Ballfield",03/05/2018,2018
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Queens,1,100040158.0,Q012-DF0750,Q-11,"C, Just Outside Playground",05/01/2018,2018
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,Desconhecido,11/30/1899 12:00:00 AM +0000,Desconhecido,X148L1,Virginia Park,Bronx,1,100040464.0,X148L1-DF0470,X-09,A,11/30/1899,1899
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,Manhattan,1,100039941.0,M144-DF0371,M-03,"E Wheelchair, Near Ballfield",04/04/2018,2018


In [64]:
spark.sql(

    """
    CREATE EXTERNAL TABLE Projeto.DrinkingFountains (
    
        FountainTy VARCHAR (50),
        the_geom VARCHAR (70),
        OBJECTID VARCHAR (50),
        Position VARCHAR (50),
        Collection VARCHAR (50),
        Painted VARCHAR (5),
        GISPROPNUM VARCHAR (50),
        SIGNNAME VARCHAR (50),
        FountainCo INT,
        GISOBJID INT,
        SYSTEM VARCHAR (50),
        DEPARTMENT VARCHAR (50), 
        DESCRIPTIO VARCHAR (50),
        Data DATE
        
        )
        
        USING DELTA
        PARTITIONED BY (
         BOROUGH VARCHAR (50),
         Year INT
       

    )
    LOCATION 'hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/DrinkingFountains'
    """
)

DataFrame[]

In [63]:
spark.sql(
    """
    DROP TABLE IF EXISTS Projeto.DrinkingFountains
    """
)

DataFrame[]

In [65]:
replaced_Fontes8 \
    .select("FountainTy", "the_geom", "OBJECTID", "Position", "Collection", "Painted", "GISPROPNUM", "SIGNNAME", "FountainCo", "GISOBJID", "SYSTEM", "DEPARTMENT", "DESCRIPTIO", "Data", "Year", "BOROUGH") \
    .write \
    .mode("overwrite") \
    .partitionBy("BOROUGH", "Year") \
    .format("delta") \
    .save("hdfs://hdfs-nn:9000/Projeto/Silver/Projeto.db/DrinkingFountains/deltalake_table/")

In [61]:
replaced_Fontes8.toPandas()

,FountainTy,the_geom,OBJECTID,Position,Collection,Painted,GISPROPNUM,SIGNNAME,BOROUGH,FountainCo,GISOBJID,SYSTEM,DEPARTMENT,DESCRIPTIO,Data,Year
0,F High Low,POINT (-73.98659181365889 40.60753207315604),1589,Out in Open,02/10/2018 12:00:00 AM +0000,no,B100,Seth Low Playground/ Bealin Square,Brooklyn,2,100038957.0,B100-DF0647,B-11,"F High Low, Out in Open",02/10/2018,2018
1,C,POINT (-73.96862765692859 40.7480871896291),1921,In Playground,02/22/2018 12:00:00 AM +0000,no,M158,Robert Moses Playground,Manhattan,1,100039303.0,M158-DF0068,M-06,"C, In Playground",02/22/2018,2018
2,D,POINT (-74.0003631604654 40.750036026114756),2253,"Under Tree, Near Ballfield, Just Outside Playg...",02/27/2018 12:00:00 AM +0000,yes,M011,Chelsea Park,Manhattan,1,100039632.0,M011-DF0209,M-04,"D, Under Tree, Near Ballfield, Just Outside Pl...",02/27/2018,2018
3,D,POINT (-73.97290101781864 40.72386196549101),2585,"Just Outside Playground, Near Ballfield",04/04/2018 12:00:00 AM +0000,no,M144,John V. Lindsay East River Park,Manhattan,1,100039936.0,M144-DF0392,M-03,"D, Just Outside Playground, Near Ballfield",04/04/2018,2018
4,C,POINT (-73.94639259466328 40.69933443616386),2917,"In Shade, In Playground",05/29/2018 12:00:00 AM +0000,Desconhecido,B302,Charlie's Place,Brooklyn,1,100040304.0,B302-DF0897,B-03,"C, In Shade, In Playground",05/29/2018,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,D,POINT (-73.90020355545877 40.62611516481206),2418,"Out in Open, Near Ballfield",03/05/2018 12:00:00 AM +0000,no,B018,Canarsie Park,Brooklyn,1,100039874.0,B018-DF0817,B-18,"D, Out in Open, Near Ballfield",03/05/2018,2018
3116,C,POINT (-73.7688486244796 40.77023172437061),2750,Just Outside Playground,05/01/2018 12:00:00 AM +0000,no,Q012,Crocheron Park,Queens,1,100040158.0,Q012-DF0750,Q-11,"C, Just Outside Playground",05/01/2018,2018
3117,A,POINT (-73.86256527267528 40.83342686523247),3082,Desconhecido,11/30/1899 12:00:00 AM +0000,Desconhecido,X148L1,Virginia Park,Bronx,1,100040464.0,X148L1-DF0470,X-09,A,11/30/1899,1899
3118,E Wheelchair,POINT (-73.97375555702659 40.7182972633732),2564,Near Ballfield,04/04/2018 12:00:00 AM +0000,yes,M144,John V. Lindsay East River Park,Manhattan,1,100039941.0,M144-DF0371,M-03,"E Wheelchair, Near Ballfield",04/04/2018,2018
